El problema de Maximum Clique (Máxima Camada) busca encontrar el subgrafo más grande donde todos los vértices están conectados entre sí. Este problema NP-hard se puede formular como un problema de Optimización Binaria Cuadrática No Restringida (QUBO).

La formulación QUBO para Maximum Clique generalmente busca maximizar el tamaño de la camarilla, pero como los solucionadores de D-Wave y otros optimizadores cuánticos o inspirados en lo cuántico trabajan minimizando, la función de costo se construye para que su mínimo corresponda al máximo tamaño de la camarilla.

Hamiltoniano / Función de Costo QUBO para Maximum Clique
El Hamiltoniano (función de costo) para el problema de Maximum Clique en un grafo $G=(V, E)$ es:$$H(x) = -A \sum_{v \in V} x_v + B \sum_{(u,v) \in \bar{E}} x_u x_v$$

Donde:$V$ es el conjunto de vértices y $\bar{E}$ es el conjunto de no-aristas (aristas que no están en $E$) del grafo $G$.$x_v \in \{0, 1\}$ es la variable binaria: $x_v=1$ si el vértice $v$ está en la camarilla, y $x_v=0$ en caso contrario.$A$ y $B$ son coeficientes de penalización/costo, elegidos de tal manera que $B$ es un valor positivo grande que actúa como penalización, y $A$ es un valor positivo que actúa como recompensa. La condición clave es $A < B$. Un valor común es $A=1$ y $B=2$ (o $B=A + \epsilon$, donde $\epsilon > 0$).

Interpretación de los Términos:
Término de Recompensa (Término Lineal, $-A$):$$-A \sum_{v \in V} x_v$$

Este término recompensa al sistema por incluir vértices en la solución.
Como estamos minimizando $H(x)$, el coeficiente negativo $-A$ impulsa al solucionador a seleccionar tantos vértices como sea posible (aumentar $\sum x_v$). Esto representa el objetivo de maximizar el tamaño de la camarilla.

Término de Restricción (Término Cuadrático, $B$):$$B \sum_{(u,v) \in \bar{E}} x_u x_v$$

Este término impone la restricción de que todos los vértices seleccionados deben estar conectados. $\bar{E}$ contiene pares de vértices $(u, v)$ que NO están conectados en $G$.Si seleccionas dos vértices $u$ y $v$ que NO están conectados ($x_u=1$ y $x_v=1$ para un par en $\bar{E}$), el término cuadrático $x_u x_v$ es 1, y se aplica una gran penalización positiva de $B$.Al minimizar $H(x)$, el algoritmo es forzado a evitar esta penalización, lo que asegura que solo se seleccionen vértices que están todos conectados entre sí (una camarilla).

**Código de Implementación en Python (Usando D-Wave Ocean SDK)**
El siguiente código utiliza la librería dimod y el solucionador de Recocido Simulado (SimulatedAnnealingSampler) para resolver el problema.

In [1]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.5/165.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00


In [3]:
!pip install dwave-neal

In [4]:
import dimod
from dimod import BQM
from neal import SimulatedAnnealingSampler
import networkx as nx

# --- 1. Definición del Grafo y Parámetros ---

# Grafo de ejemplo: Un diamante (4 vértices)
# Vértices: 0, 1, 2, 3
# Aristas: (0, 1), (0, 2), (1, 2), (2, 3)
G = nx.Graph()
G.add_edges_from([(0, 1), (0, 2), (1, 2), (2, 3)])
# La máxima camarilla es de tamaño 3: {0, 1, 2}

# Obtener el conjunto de NO-ARISTAS (E_bar) para la penalización.
# La única no-arista es (0, 3) y (1, 3).
V = G.nodes
E_bar = [(u, v) for u in V for v in V if u < v and not G.has_edge(u, v)]

# Coeficientes para el QUBO H = -A * sum(x_v) + B * sum(x_u * x_v para (u,v) en E_bar)
A = 1  # Recompensa por incluir un vértice
B = 2  # Penalización por no-aristas (debe ser B > A)

# --- 2. Construcción del Modelo QUBO (BQM) ---

bqm = BQM(dimod.Vartype.BINARY)

# 1. Término de Recompensa (Lineal): -A * sum(x_v)
# Coeficientes lineales: -A
for v in V:
    bqm.add_linear(v, -A)

# 2. Término de Restricción (Cuadrático): B * sum(x_u * x_v para (u,v) en E_bar)
# Coeficientes cuadráticos: +B
for u, v in E_bar:
    bqm.add_quadratic(u, v, B)

print("--- Modelo de Optimización (BQM/QUBO) ---")
print(bqm)

# --- 3. Ejecución del Solucionador (Recocido Simulado) ---

sampler = SimulatedAnnealingSampler()

# Ejecuta el solucionador
sampleset = sampler.sample(bqm, num_reads=100, seed=42)

# Obtiene la mejor solución (la de menor energía)
best_sample = sampleset.first.sample
best_energy = sampleset.first.energy

# --- 4. Mostrar Resultados ---

# El tamaño de la Máxima Camarilla es la suma de las variables que son 1.
clique_size = sum(best_sample.values())

print("\n--- Resultados de la Optimización ---")
print(f"Mejor Energía Encontrada (QUBO): {best_energy}")
print(f"Tamaño de la Máxima Camarilla Encontrada: {clique_size}")

# Filtrar los vértices que tienen x_v = 1
max_clique = [v for v, x in best_sample.items() if x == 1]
print(f"Vértices en la Máxima Camarilla: {max_clique}")

--- Modelo de Optimización (BQM/QUBO) ---
BinaryQuadraticModel({0: -1.0, 1: -1.0, 2: -1.0, 3: -1.0}, {(3, 0): 2.0, (3, 1): 2.0}, 0.0, 'BINARY')

--- Resultados de la Optimización ---
Mejor Energía Encontrada (QUBO): -3.0
Tamaño de la Máxima Camarilla Encontrada: 3
Vértices en la Máxima Camarilla: [0, 1, 2]


Aquí tienes la explicación detallada de la salida del Modelo QUBO (BQM) y los Resultados de la Optimización para el problema de Maximum Clique (Máxima Camada).

1. Explicación del Modelo QUBO (BQM)
La salida BinaryQuadraticModel({0: -1.0, 1: -1.0, 2: -1.0, 3: -1.0}, {(3, 0): 2.0, (3, 1): 2.0}, 0.0, 'BINARY') define la función de costo $H(x)$ para el grafo utilizado, donde $x_i \in \{0, 1\}$.La formulación utilizada es:$$H(x) = \underbrace{-A \sum_{v \in V} x_v}_{\text{Término de Recompensa}} + \underbrace{B \sum_{(u,v) \in \bar{E}} x_u x_v}_{\text{Término de Restricción}}$$
Donde se usaron los coeficientes $A=1$ y $B=2$.

A. Términos Lineales (Recompensa)
Salida: {0: -1.0, 1: -1.0, 2: -1.0, 3: -1.0}
Significado: Estos son los coeficientes $q_i$ para cada vértice $x_i$.Como $A=1$, el coeficiente lineal para cada vértice $v$ es $-A = -1.0$.
Como el solucionador busca minimizar $H(x)$, este término negativo $\mathbf{-1.0}$ actúa como una recompensa, incentivando al algoritmo a seleccionar (asignar $x_i=1$) tantos vértices como sea posible para hacer la energía más negativa.

B. Términos Cuadráticos (Restricción)Salida: {(3, 0): 2.0, (3, 1): 2.0}
Significado: Estos son los coeficientes $q_{i,j}$ para pares de vértices $(i, j)$ que NO están conectados en el grafo original (son las no-aristas $\bar{E}$).Como $B=2$, el coeficiente cuadrático es $+2.0$.Este término es una penalización.
Si se seleccionan dos vértices no conectados (por ejemplo, $x_3=1$ y $x_0=1$), se añade una penalización de $+2.0$ a la energía.
Dado que $B > A$, esta penalización supera la recompensa lineal, forzando al solucionador a evitar seleccionar pares de vértices no conectados.

Grafo Implícito:
Vértices 0, 1, 2, 3.
No-aristas: $(3, 0)$ y $(3, 1)$.
Esto implica que las aristas existentes (las de $E$) son: $(0, 1)$, $(0, 2)$, $(1, 2)$, $(2, 3)$.
El vértice 3 solo está conectado a 2.

2. Explicación de los Resultados de la OptimizaciónMejor Energía Encontrada (QUBO): -3.0Significado:
Este es el valor mínimo de la función de costo $H(x)$ que se encontró.Verificación:
La solución óptima es $x=\{1, 1, 1, 0\}$, es decir, seleccionar $\{0, 1, 2\}$.
Término de Recompensa: $-1 \times (x_0 + x_1 + x_2 + x_3) = -1 \times (1 + 1 + 1 + 0) = \mathbf{-3.0}$.Término de Restricción: Como la solución $\{0, 1, 2\}$ no contiene ninguna de las no-aristas $(3, 0)$ ni $(3, 1)$, la penalización es $0.0$.Energía Total: $-3.0 + 0.0 = \mathbf{-3.0}$.
Conclusión: La energía de $-3.0$ confirma que el algoritmo encontró una solución donde las restricciones se satisfacen perfectamente (penalización $0.0$) y la recompensa lineal es máxima para esa configuración.

**Tamaño de la Máxima Camarilla Encontrada: 3**
Significado: Este es el resultado buscado: el mayor subconjunto de vértices mutuamente conectados tiene un tamaño de 3.

**Vértices en la Máxima Camarilla: [0, 1, 2]**
Significado: Los vértices $0, 1, 2$ forman el subgrafo más grande donde todos están conectados entre sí.Verificación:
En el grafo implícito, las aristas $(0, 1)$, $(0, 2)$ y $(1, 2)$ existen, confirmando que $\{0, 1, 2\}$ es una camarilla.
Dado que el vértice 3 solo está conectado a 2, no podría formar una camarilla de tamaño 4 con el resto, por lo que el tamaño 3 es de hecho el máximo.